# Electric Lake
Electric lake is a tool to extract and cache market data



In [1]:
import requests as re
import time, json, io
from os.path import exists
from datetime import timedelta
from datetime import datetime
from math import floor
import gc

In [2]:
import pandas as pd
import numpy as np

In [3]:
from multiprocessing.pool import ThreadPool
import multiprocessing as mp

### Check for existence of the pickle

In [4]:
ticker_pickle_read = False
data_pickle_read = False
if exists("ticker_pickle.json"):
    ticker_pickle_read = True
    ticker_listing = json.load(io.open("ticker_pickle.json"))
    
if exists("data_pickle.json"):
    data_pickle_read = True
    ticker_db = json.load(io.open("data_pickle.json"))

In [5]:
if ticker_pickle_read:
    print("ticker_listing is an array with the following schema")
    print(ticker_listing['ticker' == 'A'])

if data_pickle_read:
    print("ticker_db has following schema")
    print("dict with tickers as keys")
    print("value is an array with")
    print("high, low, close, open, volume, vol-weighted price, num transactions, time represented as nansec date")
    # HACK: use agilent for last trade date, for simplicity
    print(pd.to_datetime(ticker_db['A'][-1]['t'], unit="ms" ))
    print((pd.to_datetime(ticker_db['A'][-1]['t'], unit="ms").to_pydatetime() + timedelta(days=1)).strftime("%Y-%m-%d")) # yyyy-mm-dd
    last_day_in_set = (pd.to_datetime(ticker_db['A'][-1]['t'], unit="ms").to_pydatetime() + timedelta(days=1)).strftime("%Y-%m-%d")
    current_date = datetime.today().strftime("%Y-%m-%d")
    print(current_date)

ticker_listing is an array with the following schema
{'ticker': 'A', 'name': 'Agilent Technologies Inc.', 'market': 'stocks', 'locale': 'us', 'primary_exchange': 'XNYS', 'type': 'CS', 'active': True, 'currency_name': 'usd', 'cik': '0001090872', 'composite_figi': 'BBG000BWQYZ5', 'share_class_figi': 'BBG001SCTQY4', 'last_updated_utc': '2021-12-10T00:00:00Z'}
ticker_db has following schema
dict with tickers as keys
value is an array with
high, low, close, open, volume, vol-weighted price, num transactions, time represented as nansec date
2021-12-10 05:00:00
2021-12-11
2022-03-25


### Get ticker listing; ~~pause for 12s~~

In [ ]:
if not ticker_pickle_read:
    ticker_listing = []
    
    resp = re.get("https://api.polygon.io/v3/reference/tickers?active=true&sort=ticker&order=asc&limit=1000&apiKey=3Vkvu8G6DXjilpU75icHtY_wCwwOgOjh")
    print(resp)
    #time.sleep(12)

api_key_param = "&apiKey=SANITIZED"


In [7]:
if not ticker_pickle_read:
    print(dir(resp))
    print()
    print(resp.json()['next_url'] + api_key_param)

    ticker_listing += resp.json()['results']

    print("First set of tickers: ", end='')
    for t in ticker_listing[:10]:
        print(t['ticker'], end=', ')
    print()

### Get all tickers

In [8]:
if not ticker_pickle_read:
    while resp.json().get('next_url') != None:
        # Get next url
        resp = re.get(resp.json()['next_url'] + api_key_param)

        time.sleep(12)

        ticker_listing += resp.json()['results']

        for t in resp.json()['results'][0:10]:
            print(t['ticker'], end=', ')
        print()


### Write ticker symbols to disk

In [9]:
if not ticker_pickle_read:
    pck = io.open("ticker_pickle.json", "w")
    pck.write(json.dumps(ticker_listing))
    pck.close()

### Display statistics about the types of tickers, listed exchanges, to narrow down our selection

In [10]:
ticker_stats = {'type' : {}, 'locale' : {}, 'primary_exchange' : {}}        
print(ticker_listing[0].keys())
for t in ticker_listing:
    for stat_k in ticker_stats.keys():
        if t.get(stat_k):
            if not t[stat_k] in ticker_stats[stat_k]:
                ticker_stats[stat_k] |= { t[stat_k] : 1 }
            else:
                ticker_stats[stat_k][t[stat_k]] += 1
        else:
            if not 'blank' in ticker_stats[stat_k]:
                ticker_stats[stat_k] |= { 'blank' : 1 }
            else:
                ticker_stats[stat_k]['blank'] += 1
                
ticker_stats

dict_keys(['ticker', 'name', 'market', 'locale', 'primary_exchange', 'type', 'active', 'currency_name', 'cik', 'composite_figi', 'share_class_figi', 'last_updated_utc'])


{'type': {'CS': 6807,
  'ETF': 2818,
  'blank': 2045,
  'UNIT': 783,
  'WARRANT': 1020,
  'ADRC': 533,
  'PFD': 671,
  'FUND': 472,
  'RIGHT': 42,
  'ETN': 100},
 'locale': {'us': 13616, 'global': 1675},
 'primary_exchange': {'XNYS': 4371,
  'ARCX': 1956,
  'XNAS': 6361,
  'XASE': 351,
  'BATS': 575,
  'blank': 1677}}

### Collect interesting tickers

In [11]:
interesting_ticker_symbols = []

for t in ticker_listing:
    if t.get('primary_exchange') and t.get('locale') == 'us':
        interesting_ticker_symbols += [t['ticker']]
        
print(interesting_ticker_symbols)
        

['A', 'AA', 'AAA', 'AAAU', 'AAC', 'AAC.U', 'AAC.WS', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAIC', 'AAICpB', 'AAICpC', 'AAIN', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAWW', 'AAXJ', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEQ', 'ABEV', 'ABG', 'ABGI', 'ABIO', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABRpD', 'ABRpE', 'ABRpF', 'ABSI', 'ABST', 'ABT', 'ABTX', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAQ', 'ACAQ.U', 'ACAQ.WS', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACBI', 'ACC', 'ACCD', 'ACCO', 'ACDI.U', 'ACEL', 'ACER', 'ACES', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLN', 'ACGLO', 'ACH', 'ACHC', 'ACHL', 'ACHR', 'ACHR.WS', 'ACHV', 'ACI', 'ACII', 'ACII.U', 'ACII.WS', 'ACIO', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACKIW', 'ACLS', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACPpA', 'ACQR', 'ACQRU', 'ACQRW', 'ACR', 'ACRE', 'ACRO', 'ACRO.U', 'ACRO.WS', 'ACRS', 'ACRX', 'AC

### Collect dailies

In [12]:
if not data_pickle_read:    
    unformatted_agg_url = "https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/2000-12-10/2021-12-10?adjusted=true&sort=asc&limit=50000&apiKey=3Vkvu8G6DXjilpU75icHtY_wCwwOgOjh"
    ticker_db = {}
    for t in interesting_ticker_symbols:
        resp = re.get(unformatted_agg_url.format(ticker=t))
        print("ticker:", resp.json()['ticker'], end=' ')
        print("trading days:", resp.json()['resultsCount'])
        if resp.json()['resultsCount'] > 0:
            ticker_db[t] = resp.json()['results']
        else:
            ticker_db[t] = []


# temporarily disabled to test sanity of the merge code
if data_pickle_read:    
    unformatted_agg_url = "https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{last_day_in_set}/{current_date}?adjusted=true&sort=asc&limit=50000&apiKey=3Vkvu8G6DXjilpU75icHtY_wCwwOgOjh"
    
    def _get_and_aggregate_ticker(ticker):
        local_resp = re.get(unformatted_agg_url.format(ticker=ticker,last_day_in_set=last_day_in_set,current_date=current_date))        
        print(local_resp.json()['ticker'], end=',')
        if local_resp.json()['resultsCount'] > 0:
            ticker_db[ticker] += local_resp.json()['results']
        
        if ticker=="QQQ":
            print(local_resp.json()['results'])
        
        del local_resp
        return True
    
    pool = ThreadPool(processes=8)
    
    res = [pool.apply_async(_get_and_aggregate_ticker,args=(t,)) for t in ticker_db.keys()]
    pool.close()
    pool.join()
    

AAC,AACG,AAC.WS,AAAU,AA,AAA,A,AAC.U,AACI,AACIU,AAICpB,AADI,AACIWAAIC,,AADRAAICpC,,AAME,AAIN,AAL,AAMC,AAON,AAOI,AAP,AAN,AAQC,AAPL,AAQC.UAAT,,AAQC.WS,AATC,AAU,AAWW,AAXJ,AB,ABB,ABC,ABCL,ABBV,ABCM,ABCB,ABEO,ABEQ,ABIO,ABEV,ABGI,ABM,ABG,ABMD,ABNBABOS,,ABR,ABRpD,ABRpE,ABRpF,ABSI,ABST,ABTXABT,,ABUSABVC,,AC,ACA,ACAD,ACAH,ACAHW,ACAQ.WS,ACAHU,ACAQ.U,ACAQ,ACB,ACBA,ACBAU,ACBAW,ACC,ACBI,ACCDACCO,,ACDI.U,ACEL,ACER,ACET,ACES,ACGL,ACEVW,ACEV,ACEVU,ACGLN,ACGLO,ACH,ACHC,ACHV,ACHR.WS,ACHR,ACHL,ACI,ACII,ACKIT,ACIU,ACIW,ACKIU,ACII.WS,ACII.U,ACKIW,ACIO,ACM,ACLS,ACNB,ACN,ACOR,ACP,ACMR,ACPpA,ACQR,ACRO,ACQRU,ACRO.WS,ACQRW,ACR,ACRO.U,ACRE,ACRS,ACST,ACRpD,ACRX,ACRpC,ACSI,ACT,ACTD,ACTDW,ACTG,ACTV,ACTDU,ACV,ACU,ACVFACVA,,ACXP,ACWF,ACWI,ACWV,ACY,ACWX,ADAG,ADALU,ADER,ADAP,ADBE,ADERU,ADCT,ADCpA,ADERW,ADC,ADEXADEX.U,,ADGI,ADFI,ADEX.WS,ADIL,ADES,ADI,ADIV,ADME,ADN,ADILW,ADNT,ADMA,ADM,ADMP,ADP,ADNWW,ADOC,ADOCR,ADOCW,ADRA,ADRA.U,ADPT,ADSADRA.WS,,ADRE,ADTADSK,,ADTN,ADUS,ADTX,ADVWW,ADV,AE,ADVMADXN,,ADX,ADXS,AEAC,AEACU,AEACW,

BTX,BUD,BTWNW,BTZ,BUDX,BUFD,BUFF,BUFG,BUFTBUG,,BUI,BUFR,BUL,BULZ,BUR,BURL,BUYZ,BUSE,BV,BUZZ,BVH,BVN,BVSBVXV,,BW,BWA,BWAC,BWACU,BWACW,BWAY,BWB,BWBBP,BWC,BWCAU,BWCAW,BWEN,BWFG,BWG,BWMN,BWMX,BWSN,BWX,BWXT,BWZ,BWpA,BXC,BX,BXMT,BXMX,BXP,BXRX,BXSL,BY,BYFC,BYD,BYM,BYLD,BYND,BYRN,BYTE,BYSI,BYTS,BYTSU,BZ,BZFD,BYTSW,BZFDW,BZQ,BZH,C,BZUN,CAAP,CAAS,CABA,CABO,CAC,CACC,CACI,CADL,CACG,CADEpA,CADE,CAE,CAF,CAG,CAL,CALA,CAJ,CAKE,CALF,CAH,CALB,CALM,CALX,CALT,CAMP,CAN,CAMT,CANFCANE,,CANG,CANO,CAPECAPL,,CAPRCANO.WS,,CAR,CARE,CARA,CAS,CARG,CARS,CAS.U,CARR,CARV,CARZ,CAS.WS,CASA,CASH,CASS,CASI,CASY,CAT,CATC,CATH,CATY,CATO,CB,CBAN,CBATCBAY,,CBFV,CBD,CBH,CBL,CBLSCBIO,,CBONCBNK,,CBOE,CBRE,CBRGU,CBSE,CBSH,CBRL,CBT,CBTX,CBTG,CBU,CBZ,CCAC,CC,CCAC.WS,CCAC.U,CCAIW,CCAI,CCAIU,CCAP,CCCC,CCB,CCCS,CCBG,CCCS.WSCCEL,,CCD,CCEP,CCF,CCK,CCI,CCJ,CCLP,CCM,CCL,CCNE,CCNC,CCMP,CCNEP,CCO,CCOR,CCOI,CCRN,CCRV,CCU,CCV,CCSI,CCS,CCTSU,CCV.U,CCV.WS,CCVI,CCZCCXI,,CD,CDAK,CCVI.U,CCVI.WS,CDAQ,CDAQU,CDAQWCDCCDAYCDE,,,,CDK,CDL,CDEV,CDLX,CDREC

ENJ,ENLV,ENLC,ENNV,ENNVU,ENO,ENNVW,ENOB,ENOR,ENPC,ENPC.U,ENPC.WS,ENPH,ENR,ENRpA,ENSC,ENS,ENSG,ENSV,ENTA,ENTFU,ENTFW,ENTF,ENTR,ENTG,ENTX,ENTXW,ENV,ENVA,ENVIU,ENVI,ENVB,ENVIW,ENVX,ENX,ENVXW,EOCT,EOCWENZL,,ENZ,EOCW.U,EODEOCW.WS,,EOG,EOI,EOSEOPS,,EOLS,EOSE,EOT,EPAC,EOSEW,EPD,EPAY,EPCEPAM,,EPHE,EPHYU,EPHY,EPHYW,EPP,EPM,EPIX,EPOL,EPI,EPRF,EPR,EPRE,EPRT,EPRpC,EPRpE,EPRpG,EPS,EPSN,EPU,EPV,EPWR,EPWR.WS,EPWR.U,EPpC,EPZM,EQ,EQBKEQAL,,EQDEQD.WS,,EQCpD,EQC,EQD.U,EQHA,EQHA.U,EQH,EQHpC,EQHpA,EQHA.WS,EQL,EQIX,EQNR,EQOS,EQOP,EQRR,EQT,EQS,EQR,EQUL,EQWL,EQX,ERAS,ERC,ERESU,ERESW,ERES,ERF,ERH,ERIC,ERII,ERIE,EROERM,,ERJ,ERSX,ERTH,ERUS,ERX,ERY,ERYP,ES,ESACU,ESBA,ESCA,ESCRESBK,,ESE,ESEA,ESEB,ESG,ESGA,ESGB,ESGD,ESGC,ESGE,ESGNESGG,,ESGRO,ESGR,ESGRP,ESGS,ESGV,ESGU,ESGYESI,,ESHY,ESLT,ESM,ESM.U,ESML,ESM.WS,ESMV,ESP,ESMT,ESNT,ESPO,ESRT,ESPR,ESQ,ESSC,ESS,ESSA,ESSCR,ESSCU,ESTC,ESSCW,ESTA,ESUS,ESTE,ETAC,ET,ETACU,ETB,ETD,ETACW,ETG,ETIp,ETJ,ETHO,ETNB,ETO,ETN,ETON,ETR,ETRN,ETTX,ETSY,ETWO.WSETV,,ETWETWO,,ETX,ETY,ETpC,ETpD

HHGCW,HHLA,HHLA.U,HHLA.WSHHH,,HHR,HHSHI,,HIBB,HIBL,HIBS,HIG,HIFS,HIGA,HIE,HIGA.U,HIGA.WS,HIGpG,HIHO,HIII,HIIIW,HIIIUHII,,HIL,HIMX,HIMS,HIO,HITI,HIPO,HIPO.WS,HIPS,HIVE,HIW,HIX,HJPX,HJEN,HKNDHL,,HKIB,HLAH,HLAHU,HLAHW,HLBZ,HLAL,HLGHLGE,,HLF,HLBZW,HLI,HLIO,HLLY,HLLY.WS,HLIT,HLMN,HLMNW,HLNE,HLT,HLTH,HLX,HMC,HMCO,HLXA,HLpB,HMCOU,HMCOW,HMG,HMHC,HMLPHMLPpA,HMN,,HMNF,HMOP,HMPT,HMST,HMTV,HNDL,HNI,HNNA,HNGR,HMY,HNNAZ,HNP,HNRG,HOFV,HNST,HOFT,HNWHOFVW,,HOG,HOLI,HOLD,HOMB,HOLX,HOMZ,HON,HOM,HONE,HOOD,HOOK,HOPE,HORIU,HOTH,HOV,HOVNP,HOWL,HP,HPE,HPI,HPF,HPK,HPKEW,HPLTU,HPLT,HPLTW,HPP,HPQ,HPPpC,HPS,HPX,HPX.WS,HPX.U,HQH,HQL,HQY,HR,HRB,HQI,HRC,HRI,HRL,HROW,HRMY,HRT,HROWL,HRTG,HRTX,HRZN,HSAQ,HSC,HSCZ,HSDT,HSIC,HSBC,HSII,HSKA,HSMV,HSON,HST,HSRT,HSTM,HSTO,HSUN,HSY,HT,HTAQ.U,HTAB,HTBI,HTBK,HTA,HTBX,HTD,HTEC,HTGC,HTFB,HTHT,HTH,HTGM,HTIA,HTIBP,HTLD,HTLFP,HTOO,HTLF,HTOOW,HTPA.U,HTPA,HTPA.WS,HTUS,HTY,HTZWW,HTZ,HTRB,HTpC,HTpD,HTpE,HUBG,HUBB,HUGE,HUBS,HUDI,HUGS,HUGS.U,HUGS.WS,HUIZ,HUM,HUMAW,HUN,HUMA,HURN,HURC,HUSA,H

MAS,MASI,MATMASS,,MATW,MAVMATX,,MAX,MAXR,MAXN,MAYZMAYS,,MBAC.WS,MBAC,MBAC.U,MBB,MBBB,MBCC,MBI,MBCN,MBINN,MBIN,MBII,MBINO,MBINP,MBIO,MBNDMBNKP,,MBOT,MBRXMBOX,,MBSC.U,MBT,MBSD,MBTCR,MBTCMBTCU,,MBWM,MBUU,MC,MCAAU,MCA,MCAF,MCAEUMCAER,,MCAFRMCAE,,MCAFU,MCAG,MCAGR,MCAGU,MCBSMCBC,,MCD,MCB,MCEF,MCFE,MCFT,MCHP,MCHX,MCI,MCHI,MCG,MCK,MCLD,MCMJ,MCR,MCNMCO,,MCRB,MCMJW,MCRI,MCRO,MCS,MCW,MCY,MD,MDB,MDC,MDCP,MDEV,MDGL,MDH,MDGS,MDGSW,MDH.U,MDH.WS,MDIA,MDIV,MDJH,MDNA,MDRR,MDLZ,MDRRP,MDRX,MDT,MDU,MDVA,MDXG,MDXH,MDVLMDWD,,MDWT,MDYG,MDY,MDYV,ME,MEAC,MECMEAR,,MEACW,MEACU,MED,MEDP,MEDS,MEGI,MELI,MEI,MEIP,MEKA,MEG,MEME,MENV,MEOA,MERpK,MEOAW,MERC,MEOAU,MEOH,MESA,MESO,META,METXW,METCL,MET,METC,METXMETpA,,METpE,MEUSW,METpF,MEXX,MF,MFA,MFApC,MFApB,MFC,MFG,MFD,MFDX,MFEM,MFH,MFIN,MFGP,MFL,MFM,MFUL,MFMS,MFUS,MFV,MG,MGA,MGC,MGEE,MGIC,MGIMGF,,MGK,MGM,MGLN,MGMT,MGNI,MGNX,MGPI,MGR,MGP,MGRB,MGRC,MGRD,MGTX,MGTA,MGU,MGV,MGY,MHD,MGYR,MHF,MHH,MHI,MHK,MHLA,MHNC,MHN,MHLD,MHO,MHpA,MHpC,MHpD,MIC,MICT,MID,MIDD,MIDE,MIDF,MIG,MIGI,

PNFP,PNFPP,PNI,PNM,PNNT,PNOV,PNQI,PNR,PNT,PNRG,PNTM,PNTM.U,PNTG,PNTM.WS,PNW,POAI,POCTPODD,,POLA,POLY,POND,POND.WS,POND.U,PONO,PONOW,PONOU,POOL,POR,PORT.U,POSH,POTX,POST,POW,POWI,POWL,POWRU,POWRW,POWW,PPA,PPBI,POWWP,PPC,PPGHPPBT,,PPG,PPGHU,PPGHW,PPH,PPHP,PPHPR,PPHPUPPHPW,,PPL,PPIH,PPLT,PPSI,PPT,PPTA,PQINPQDI,,PRAPPTY,,PRAA,PRAX,PRBM,PRBM.U,PRCH,PRCT,PRBM.WS,PRDO,PREF,PREpJ,PRF,PRFZ,PRFT,PRFX,PRG,PRGO,PRGS,PRI,PRIFpD,PRIFpF,PRIFpG,PRIFpH,PRIFpI,PRIFpK,PRIM,PRIFpJ,PRK,PRLB,PRM,PRLD,PRN,PRMW,PRNTPRO,,PROC,PROF,PROG,PROCW,PRPB,PROV,PRPB.WS,PRPCPRPB.U,,PRPC.U,PRPH,PRPC.WS,PRPL,PRPO,PRQR,PRS,PRSR,PRSRU,PRSRW,PRT,PRTA,PRTC,PRTG,PRTH,PRTK,PRTS,PRTY,PRU,PRVA,PSA,PRVB,PSAGU,PSAG,PSAGW,PSApE,PSApF,PSApG,PSApI,PSApH,PSApK,PSApJ,PSApMPSApL,,PSApO,PSApQ,PSApP,PSApN,PSApR,PSB,PSBpXPSBpY,,PSCD,PSBpZPSC,,PSCC,PSCE,PSCF,PSCI,PSCT,PSCJ,PSCWPSCQ,,PSCU,PSCH,PSCM,PSCX,PSDN,PSECpA,PSEP,PSF,PSEC,PSET,PSFD,PSFE,PSFF,PSFE.WS,PSFJ,PSFO,PSFMPSHG,,PSI,PSIL,PSLV,PSL,PSJ,PSK,PSMB,PSMC,PSMD,PSMG,PSMJ,PSMO,PSMM,PSMT,PS

QQQE,QQQJ,QQQM,QQQX,QQQA,QQQN,QQXT,QRFT,QRHC,QRMI,QRTEA,QRTEPQRTEB,,QRVO,QSIQS,,QSIAW,QSPT,QTAP,QSWN,QSYQSR,,QTEC,QTJL,QTNT,QTUM,QTOC,QTT,QTR,QTRX,QTWO,QUAD,QUAL,QUBT,QULLQUIK,,QUMU,QUOT,QURE,QUS,QVAL,QVCC,QVML,QVCD,QVMS,QVMM,QWLD,QYLD,QYLG,R,RAAS,RAAX,RACBRA,,RACE,RADA,RAD,RADI,RAFE,RAIL,RAIN,RALS,RAMMU,RAMMW,RAM,RAMP,RAND,RARE,RAPT,RANI,RAVI,RAYS,RAVERAYC,,RBA,RBAC.U,RBAC,RBAC.WS,RBCAARBB,,RBBN,RBCN,RBIN,RBKB,RBNC,RBLX,RBND,RBOT,RBOT.WSRBUS,,RC,RCA,RCB,RCAT,RCELRCCRCD,,,RCFA.U,RCHG,RCG,RCHGU,RCHGW,RCII,RCKTRCI,,RCKY,RCL,RCLF,RCLFW,RCLFU,RCM,RCMT,RCON,RCOR.WS,RCRT,RCOR,RCSRCRTW,,RCUS,RCpE,RCpC,RDBX,RDBXW,RDCM,RDFI,RDFN,RDI,RDHL,RDIB,RDNRDNT,,RDOG,RDS.A,RDS.B,RDUSRDVT,,RDVY,RDW.WS,RDW,RDY,RDWR,RE,REAXREC,,REAL,RECS,REDU,REEAW,REE,REED,REFRREFI,,REET,REG,REGN,REI,REGI,REGL,REK,REIT,REKR,RELI,RELIW,RELX,RELL,RELY,REMX,REML,REM,RENN,RENT,REPH,REPL,REPX,RERE,RES,RESD,RESE,RESN,RETA,RESP,RETL,RETO,REV,REVE,REVEU,REVEW,REVH,REVG,REVHU,REVHW,REVS,REW,REX,REXR,REXRpC,REXRpB,REYN,REZ,RF,RFCI,RE

TMKRW,TMO,TMKRUTMP,,TMPM,TMPMU,TMQ,TMPMW,TMST,TMUS,TMX,TMV,TNA,TNET,TNC,TNDM,TNGX,TNK,TNPpDTNL,,TNP,TNPpE,TNPpF,TNXP,TNYA,TOACU,TOIIW,TOI,TOK,TOKE,TOLZ,TOL,TOMZ,TOPS,TOTR,TOTL,TOST,TPAYTOWN,,TOURTPB,,TPBA,TPBAUTPBAW,,TPC,TPHTPGY.WSTPGY.U,,TPGY,,TPHD,TPHE,TPHS,TPIC,TPIF,TPLC,TPLE,TPL,TPOR,TPR,TPSCTPST,,TPTA,TPTX,TPVG,TPX,TPYP,TR,TPZ,TQQQ,TRC,TRCATRAQ.U,,TRCA.WS,TRCA.U,TRDA,TREB.UTREB,,TREE,TREC,TREB.WS,TREX,TRGP,TRHC,TRIB,TRI,TRIP,TRIN,TRIS,TRIS.WSTRIS.U,,TRIT,TRITW,TRKAW,TRKATRMD,,TRMB,TRMR,TRMK,TRN,TRND,TRNO,TRONU,TRONW,TRNS,TRON,TROW,TROO,TROX,TRPL,TRP,TRST,TRS,TRQ,TRT,TRTL,TRTL.U,TRTN,TRTNpA,TRTL.WS,TRTNpB,TRTNpC,TRTNpD,TRTNpE,TRTX,TRU,TRUE,TRTXpC,TRV,TRUP,TRVG,TRTY,TRVI,TRVN,TRYP,TRXTS,,TSAT,TSBK,TSC,TSCAP,TSE,TSEM,TSCO,TSHA,TSI,TSCBP,TSIBU,TSIB,TSIBW,TSLA,TSJA,TSLX,TSM,TSN,TSP,TSOC,TSPA,TSPQ,TSQ,TSPQ.U,TSPQ.WS,TSRI,TT,TSVT,TTAC,TTAI,TTC,TTCF,TTE,TTD,TTEC,TTEK,TTGT,TTI,TTMI,TTM,TTNP,TTPTTOO,,TTSH,TTT,TUEMTTWO,,TU,TUFN,TUGCU,TUGC,TUGCW,TUP,TUR,TURN,TUSA,TUSK,TV,TUYA,TVAC,TVACU,TVE,T

In [13]:
interesting_ticker_symbols[3079]

'BBSI'

### Save the database to disk to support good differential updates

In [14]:
if not data_pickle_read:
    pck = io.open("data_pickle.json", "w")
    pck.write(json.dumps(ticker_db))
    pck.close()
    
    pck = io.open("data_pickle.json.volatile", "w")
    pck.write(json.dumps(ticker_db))
    pck.close()

In [15]:
print(len(ticker_db))
print(len(interesting_ticker_symbols))

11613
13614


In [16]:
ticker_db.keys()

dict_keys(['A', 'AA', 'AAA', 'AAAU', 'AAC', 'AAC.U', 'AAC.WS', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAIC', 'AAICpB', 'AAICpC', 'AAIN', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAWW', 'AAXJ', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEQ', 'ABEV', 'ABG', 'ABGI', 'ABIO', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABRpD', 'ABRpE', 'ABRpF', 'ABSI', 'ABST', 'ABT', 'ABTX', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAQ', 'ACAQ.U', 'ACAQ.WS', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACBI', 'ACC', 'ACCD', 'ACCO', 'ACDI.U', 'ACEL', 'ACER', 'ACES', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLN', 'ACGLO', 'ACH', 'ACHC', 'ACHL', 'ACHR', 'ACHR.WS', 'ACHV', 'ACI', 'ACII', 'ACII.U', 'ACII.WS', 'ACIO', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACKIW', 'ACLS', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACPpA', 'ACQR', 'ACQRU', 'ACQRW', 'ACR', 'ACRE', 'ACRO', 'ACRO.U', 'ACRO.WS', 'ACRS', '

### Select interesting data

In [17]:
unique_times = []
for r in ticker_db['A']:
    if not r['t'] in unique_times:
        unique_times += [r['t']]
    else:
        print(r['t'])

print(ticker_db['A'][-1])
print(ticker_db['A'][-6])

{'v': 1403735.0, 'vw': 135.1972, 'o': 137.3, 'c': 135.71, 'h': 137.66, 'l': 133.7, 't': 1648180800000, 'n': 23527}
{'v': 2766601.0, 'vw': 138.2767, 'o': 136.21, 'c': 139.12, 'h': 139.393, 'l': 136.21, 't': 1647576000000, 'n': 25832}


### Convert into a Pandas dataframe, similar enough to the old one not to have to rebuild TR1007

In [18]:
all_ticker_dfs = []
gc.collect()
for t in ticker_db.keys():
    if ticker_db[t]:
        curdf = pd.DataFrame.from_dict(ticker_db[t], dtype=np.float64)
        curdf.set_index('t', inplace=True)        
        curdf = curdf.add_suffix('_{}'.format(t))
        curdf = curdf.rename(mapper={'vw_{}'.format(t): 'Close_{}'.format(t)},axis='columns')
        all_ticker_dfs += [curdf]

del ticker_db

In [19]:
all_ticker_dfs[0].describe()

,v_A,Close_A,o_A,c_A,h_A,l_A,n_A
count,4.655000e+03,4655.000000,4655.000000,4655.000000,4655.000000,4655.000000,4655.000000
mean,3.474047e+06,45.415282,45.407828,45.417285,45.884363,44.931101,15415.186037
std,2.281332e+06,33.997465,34.016175,33.994478,34.317134,33.680805,9336.003842
min,2.816050e+05,9.002500,8.908900,8.894600,9.280700,8.594300,918.000000
25%,1.946922e+06,23.470400,23.487700,23.519950,23.798750,23.137400,9584.000000
50%,2.975385e+06,31.944300,31.974800,31.982000,32.210800,31.631600,14568.000000
75%,4.369309e+06,61.245600,61.265000,61.195000,61.620000,60.760000,19886.000000
max,3.114503e+07,179.021000,179.280000,179.280000,179.570000,177.170000,99980.000000


In [20]:
# join all collected DFs
gc.collect()
big_pickle = all_ticker_dfs[0].join(all_ticker_dfs[1:])

# free memory
#del all_ticker_dfs

In [21]:
# TODO: introduce a feature that captures this information instead
# TODO: evaluate backfill vs zero vs large, etc.
big_pickle = big_pickle.fillna(0)

In [22]:
big_pickle.index= pd.to_datetime(big_pickle.index, unit='ms')

In [23]:
big_pickle

,v_A,Close_A,o_A,c_A,h_A,l_A,n_A,v_AA,Close_AA,o_AA,...,h_ZYNE,l_ZYNE,n_ZYNE,v_ZYXI,Close_ZYXI,o_ZYXI,c_ZYXI,h_ZYXI,l_ZYXI,n_ZYXI
t,,,,,,,,,,,,,,,,,,,,,
2003-09-11 04:00:00,2.902657e+06,17.7711,17.5676,17.9036,18.0037,17.4817,1907.0,1.217700e+06,84.7560,85.20,...,0.0000,0.0000,0.0,0.0,0.0000,0.00,0.00,0.000,0.00,0.0
2003-10-01 04:00:00,3.337063e+06,15.9460,15.8373,15.9088,16.1018,15.7586,1940.0,2.736200e+06,79.7100,78.90,...,0.0000,0.0000,0.0,0.0,0.0000,0.00,0.00,0.000,0.00,0.0
2003-10-02 04:00:00,3.371608e+06,15.7329,15.8015,15.7228,15.8587,15.6156,2192.0,1.260133e+06,81.3612,80.64,...,0.0000,0.0000,0.0,0.0,0.0000,0.00,0.00,0.000,0.00,0.0
2003-10-03 04:00:00,5.526294e+06,16.4271,16.5165,16.4450,16.7810,15.8873,2341.0,2.238600e+06,84.2301,81.39,...,0.0000,0.0000,0.0,0.0,0.0000,0.00,0.00,0.000,0.00,0.0
2003-10-06 04:00:00,1.322517e+06,16.4885,16.4450,16.4665,16.6095,16.3092,1680.0,1.108533e+06,85.0908,85.44,...,0.0000,0.0000,0.0,0.0,0.0000,0.00,0.00,0.000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-21 04:00:00,1.285102e+06,138.0140,137.6300,138.1400,139.7900,137.0100,26403.0,9.901007e+06,90.1853,85.57,...,2.0600,1.9400,2310.0,299396.0,5.7722,5.78,5.78,5.965,5.64,4342.0
2022-03-22 04:00:00,1.977714e+06,138.9244,138.0100,139.2900,140.1997,137.6500,25031.0,6.100540e+06,89.7857,89.98,...,2.0950,1.9101,2242.0,346504.0,5.9702,5.83,5.98,6.120,5.83,3970.0
2022-03-23 04:00:00,1.337390e+06,134.7575,137.8500,134.1800,137.9550,133.4482,24941.0,6.848035e+06,93.9962,91.49,...,2.1100,1.9900,2593.0,517600.0,6.2789,5.98,6.25,6.520,5.91,5602.0


### Final result pickled to provide to TR1007

In [24]:
big_pickle.to_pickle("BIG.pickle")

In [25]:
i = 0
for v in big_pickle['Close_WOOD']:
    if v > 0:
        break
    i+=1
i

big_pickle['Close_WOOD'][i:]

t
2008-06-25 04:00:00    48.9286
2008-06-26 04:00:00    47.6243
2008-06-27 04:00:00    47.3764
2008-06-30 04:00:00    47.3000
2008-07-01 04:00:00    46.5432
                        ...   
2022-03-21 04:00:00    91.8730
2022-03-22 04:00:00    91.6823
2022-03-23 04:00:00    90.6818
2022-03-24 04:00:00    90.5545
2022-03-25 04:00:00    90.3678
Name: Close_WOOD, Length: 3463, dtype: float64